In [192]:
from time import sleep as s
from selenium import webdriver
from bs4 import BeautifulSoup as bs4

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

K = [
    'A','B','C','D','E',
    'F','G','H','I','J',
    'K','L','M','N','O',
    'P','Q','R','S','T',
    'U','V','W','X','Y',
    'Z','#','~',
]

In [193]:
driver = webdriver.Firefox()
btt_xpath = '/html/body/div/div[3]/div[2]/div/div/div[8]/a[3]'
firs_row_col_xpath = '/html/body/div/div[3]/div[2]/div/div/div[5]/table/tbody/tr[1]/td[1]/a' 

for k in K:
    
    bands_list = []
    is_first_iter = True
    url = f'https://www.metal-archives.com/lists/{k}'

    driver.get(url)

    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.XPATH, firs_row_col_xpath)))

    while True:

        html = bs4(
            driver.page_source,
            "html.parser"
        )

        bands_info = [c.text for r in html.find_all('tr') for c in r.find_all('td')]

        if is_first_iter:
            is_first_iter = False
            last_iter_hash = ''.join(bands_info[-4:])

        else:
            current_iter_hash = ''.join(bands_info[-4:])
            if current_iter_hash == last_iter_hash:
                break
            else:
                last_iter_hash = current_iter_hash

        bands_dict = [
            {
                'name':    bands_info[i],
                'country': bands_info[i+1],
                'genre':   bands_info[i+2],
                'status':  bands_info[i+3],
            }
            for i in range(0, len(bands_info), 4)    
        ]

        bands_list += bands_dict.copy()

        button = driver.find_element(By.XPATH, btt_xpath)
        button.click()
        s(3)
        
    pd.DataFrame(bands_list).to_csv(f'bands_{k}.csv')